[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/g_parabolic/parabolic.ipynb)

In [40]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from math import *

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

# Parabolic equations

## Heat equation

We will solve for the heat equation
$$
u_t = u_{xx}
$$
with a periodic boundary condition $0 \leq x < 1$ and initial condition
$$
u(x,0) = 
\begin{cases} 
1 & \text{if } 1/4 \leq x < 3/4, \\
0 & \text{otherwise.}
\end{cases}
$$

### Setup

In [60]:
# Grid size
m = 64
a = np.zeros((m))
b = np.empty((m))
snaps = 40
iters = 50
z = np.empty((m, snaps+1))

# PDE-related constants. Change the timestep prefactor to 0.5001 to go slightly
# beyond the point of stability, where the 2-gridpoint oscillation will slowly
# grow.
dx = 1.0/m
dt = 0.1*dx*dx
mu = dt/(dx*dx)

# Initial condition
for j in range(m):
    x = dx*j
    if x > 0.25 and x < 0.75:
        a[j] = 1
z[:, 0] = a

### Integrate the PDE

In [ ]:
# Store results
results = []

# Integrate the PDE
for i in range(1, snaps+1):
    for k in range(iters):
        for j in range(m):
            jl = j-1
            if jl < 0:
                jl += m
            jr = j+1
            if jr >= m:
                jr -= m
            b[j] = ((1-2*mu)*a[j]+mu*(a[jl]+a[jr]))
        a = np.copy(b)
    z[:, i] = a

# Output results
for j in range(m):
    e = [str(j*dx)]
    for i in range(snaps+1):
        e.append(str(z[j, i]))
    print(" ".join(e))
    results.append(" ".join(e).split())

### Visualize

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6), dpi=300)

results = np.array(results, dtype=float)
colors = plt.cm.viridis(np.linspace(0, 1, snaps+1))

for i in range(1, snaps+1):
    ax.plot(results[:, 0], results[:, i],
            label=f't={i*dt*iters:.2f}', color=colors[i])

# Formatting
ax.set_xlabel('$x$')
ax.set_ylabel('$u(x,t)$')
sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=0, vmax=snaps*dt*iters))
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax, label='Time')

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 8), dpi=300)
ax = fig.add_subplot(111, projection='3d')

# Prepare data for the 3D plot
X = np.linspace(0, 1, m)  # spatial dimension
T = np.linspace(0, snaps * dt * iters, snaps + 1)  # time dimension
X, T = np.meshgrid(X, T)
Z = z.T  # transpose z to match the dimensions of X and T

surf = ax.plot_surface(X, T, Z, cmap='inferno', edgecolor='none')

# Formatting
ax.set_xlabel('$x$')
ax.set_ylabel('$t$')
ax.set_zlabel('$u(x,t)$')
fig.colorbar(surf, ax=ax, label='Temperature', shrink=0.5)

plt.show()

## Crank–Nicolson method

The example code solves the heat equation using the Crank-Nicolson method, which is second-order in space and time, and unconditionally stable.

We have the PDE ($x \in [0,1]$ and $t \in [0, 0.1]$)
$$
u_t = u_{xx}
$$
with boundary conditions and initial conditions
$$
u(0,t) = u(1,t) = 0.
$$

This PDE has an exact solution
$$
u(x,t) = e^{-\pi^2 t}\sin(\pi x)
$$
which we will use it to compute the L2 norm.

### Setup

In [2]:
# Solves the heat equation using the Crank-Nicolson method using
# a given number of grid points, and computes the L2 norm to a reference
# solution
def c_n(m):
    a = np.empty((m+1))
    b = np.empty((m+1))

    # Set grid spacing, timestep, and mu
    dx = 1.0/m
    dt = 0.1/m
    mu = dt/(dx*dx)

    # Set initial condition
    for j in range(m+1):
        a[j] = sin(j*pi*dx)
    b[0] = 0
    b[m] = 0

    # Create linear system matrix for implicit step in Crank-Nicolson. Note
    # that here the matrix is represented and solved using dense linear
    # algebra, which is inefficient for large m. Using a dedicated tridiagonal
    # solver would be more efficient.
    A = np.diag(np.ones(m+1)*(1+mu)) + \
        np.diag(np.ones(m)*(-0.5*mu), 1) + \
        np.diag(np.ones(m)*(-0.5*mu), -1)
    A[0, 0] = 1
    A[0, 1] = 0
    A[m, m] = 1
    A[m, m-1] = 0

    # Perform timesteps
    for i in range(m):
        for j in range(1, m):
            b[j] = (1-mu)*a[j]+0.5*mu*(a[j-1]+a[j+1])
        a = np.linalg.solve(A, b)

    # Evaluate L2 norm to reference solution
    s = 0
    f = exp(-pi*pi*0.1)
    for j in range(1, m):
        du = a[j]-sin(j*pi*dx)*f
        s += du*du
    return sqrt(dx*s)

### Integrate the PDE

In [ ]:
# Store the results for plotting
results = []

# Loop over a range of grid sizes
m = 4
while m <= 512:

    # Print the number of grid points, the grid spacing, and the L2 norm
    # for Crank-Nicolson
    print(m, 1.0/m, c_n(m))
    results.append([m, 1.0/m, c_n(m)])

    # Double the number of grid points
    m *= 2

### Visualize

In [4]:
# Extract variables
m = np.array([r[0] for r in results])
dx = np.array([r[1] for r in results])
cn = np.array([r[2] for r in results])

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 4), dpi=300)
ax.plot(m, cn, marker='o')

# Formatting
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('$m$')
ax.set_ylabel('$L_2$ norm')
ax.grid(True, which="major", ls="--", linewidth=0.5)
ax.axis('equal')

plt.show()